# DML and DMR Analysis

In this notebook, I will examine the location of differentially methylated loci (DML) and regions (DMR) in the *C. virginica* genome. The DML and DMR were identified using methylKit in [this R script](https://github.com/fish546-2018/yaamini-virginica/tree/master/analyses/2018-10-25-MethylKit).

Methods:

0. Prepare for Analyses
1. Locate Files and Set Variable Paths
2. Identify Overlaps with Genomic Feature Tracks
3. Identify Overlaps between CG motifs and other Feature Tracks
4. Identify Overlaps between Transposable Elements and Other Feature Tracks
5. Calculate Overlap Proprtions
6. Gene Flanking

## 0. Prepare for Analyses

### 0a. Set Working Directory

In [1]:
pwd

'/Users/yaamini/Documents/yaamini-virginica/notebooks'

In [2]:
cd ../analyses/

/Users/yaamini/Documents/yaamini-virginica/analyses


In [1]:
!mkdir 2018-11-01-DML-and-DMR-Analysis

In [4]:
ls -F

2018-10-25-MethylKit/                2019-01-15-Sample-Clustering/
2018-11-01-DML-and-DMR-Analysis/     2019-03-07-IGV-Verification/
2018-12-02-Gene-Enrichment-Analysis/ README.md


In [3]:
cd 2018-11-01-DML-and-DMR-Analysis/

/Users/yaamini/Documents/yaamini-virginica/analyses/2018-11-01-DML-and-DMR-Analysis


### 0b. Download Genome Feature Files

I will be using the following tracks:

1. Exon: Coding regions
2. Intron: Regions that are removed
3. Genes: This includes exons and introns, as well as constituent mRNA.
4. Transposable elements (all): Transposable elements located using information all species in the RepeatMasker databse (see [Sam's notes](http://onsnetwork.org/kubu4/2018/08/28/transposable-element-mapping-crassostrea-virginica-genome-cvirginica_v300-using-repeatmasker-4-07/) for more information)
5. Tranpsosable elements (_C. gigas_): Transposable elements located using information from _C. gigas_ only (see [Sam's notes](http://onsnetwork.org/kubu4/2018/08/28/transposable-element-mapping-crassostrea-virginica-genome-cvirginica_v300-using-repeatmasker-4-07/) for more information)
4. CG motifs: Regions with CGs where methylation can occur

In [ ]:
!curl https://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2019-05-13-Yaamini-Virginica-Repository/analyses/2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_exon_sorted_yrv.bed > C_virginica-3.0_Gnomon_exon_sorted_yrv.bed

In [ ]:
!curl https://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2019-05-13-Yaamini-Virginica-Repository/analyses/2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_intron_yrv.bed > C_virginica-3.0_Gnomon_intron_yrv.bed

In [ ]:
!curl https://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2019-05-13-Yaamini-Virginica-Repository/analyses/2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_gene_sorted_yrv.bed > C_virginica-3.0_Gnomon_gene_sorted_yrv.bed

In [9]:
!curl http://owl.fish.washington.edu/halfshell/genomic-databank/C_virginica-3.0_TE-all.gff > C_virginica-3.0_TE-all.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 63.0M  100 63.0M    0     0  45.2M      0  0:00:01  0:00:01 --:--:-- 45.3M


In [10]:
!curl http://owl.fish.washington.edu/halfshell/genomic-databank/C_virginica-3.0_TE-Cg.gff > C_virginica-3.0_TE-Cg.gff

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 57.4M  100 57.4M    0     0  47.4M      0  0:00:01  0:00:01 --:--:-- 47.5M


In [ ]:
!curl https://gannet.fish.washington.edu/spartina/2018-10-10-project-virginica-oa-Large-Files/2019-05-13-Yaamini-Virginica-Repository/analyses/2019-05-13-Generating-Genome-Feature-Tracks/C_virginica-3.0_Gnomon_mRNA_yrv.bed >  C_virginica-3.0_Gnomon_mRNA_yrv.bed

In [104]:
!ls C_virginica*

C_virginica-3.0_CG-motif.bed
C_virginica-3.0_CG-motif.bed.idx
C_virginica-3.0_Gnomon_exon_sorted_yrv.bed
C_virginica-3.0_Gnomon_gene_sorted_yrv.bed
C_virginica-3.0_Gnomon_intron_yrv.bed
C_virginica-3.0_Gnomon_mRNA_yrv.bed
C_virginica-3.0_TE-Cg.gff
C_virginica-3.0_TE-all.gff


## 1. Locate Relevant Files and Set Variable Path Names

### 1a. Set Variable Path Names

Setting the variable path names allows me to reuse this script with different input files or different paths to programs without manually changing the file names each time.

In [7]:
bedtoolsDirectory = "/Users/Shared/bioinformatics/bedtools2/bin/"

In [8]:
DMLlist = "../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations.bed"

In [9]:
hyperDML = "../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations-Hypermethylated.bed"

In [10]:
hypoDML = "../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations-Hypomethylated.bed"

In [9]:
DMRlist = "../../analyses/2018-10-25-MethylKit/2018-11-07-DMR-Locations.bed"

In [11]:
exonList = "C_virginica-3.0_Gnomon_exon_sorted_yrv.bed"

In [20]:
intronList = "C_virginica-3.0_Gnomon_intron_yrv.bed"

In [13]:
geneList = "C_virginica-3.0_Gnomon_gene_sorted_yrv.bed"

In [14]:
transposableElementsAll = "C_virginica-3.0_TE-all.gff"

In [15]:
transposableElementsCg = "C_virginica-3.0_TE-Cg.gff"

In [16]:
CGMotifList = "C_virginica-3.0_CG-motif.bed"

In [105]:
mRNAList = "C_virginica-3.0_Gnomon_mRNA_yrv.bed"

### 1b. Confirm Variable Path Works and Characterize Files

The BEDfiles with DML and DMR can be viewed below. Columns are are the chromosome, start position, end position, strand, and fold difference with direction. The files only have DML and DMR that were at least 50% different between the two treatments (control and elevated pCO<sub>2</sub>).

In [14]:
#Previewing the files
!head {DMLlist}

NC_035780.1	571138	571140	58
NC_035780.1	1882691	1882693	64
NC_035780.1	1885022	1885024	61
NC_035780.1	1933499	1933501	51
NC_035780.1	1958998	1959000	50
NC_035780.1	2538924	2538926	-50
NC_035780.1	2541726	2541728	-54
NC_035780.1	2584492	2584494	56
NC_035780.1	2586508	2586510	-53
NC_035780.1	2588794	2588796	-53


In [13]:
#Counting the number of lines to count DML
!wc -l {DMLlist}

     598 ../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations.bed


In [17]:
!head {DMRlist}

NC_035780.1	571100	571200	DMR	58
NC_035780.1	573700	573800	DMR	52
NC_035780.1	1885000	1885100	DMR	51
NC_035780.1	1933500	1933600	DMR	53
NC_035780.1	4285700	4285800	DMR	-51
NC_035780.1	24159600	24159700	DMR	51
NC_035780.1	26629500	26629600	DMR	65
NC_035780.1	28563400	28563500	DMR	59
NC_035780.1	29883000	29883100	DMR	-55
NC_035780.1	31302900	31303000	DMR	-61


In [16]:
!wc -l {hyperDML}

     310 ../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations-Hypermethylated.bed


In [18]:
!head {hyperDML}

NC_035780.1	401630	401632	53
NC_035780.1	571138	571140	58
NC_035780.1	1882691	1882693	64
NC_035780.1	1885022	1885024	61
NC_035780.1	1933499	1933501	51
NC_035780.1	2584492	2584494	56
NC_035780.1	2589720	2589722	57
NC_035780.1	4286286	4286288	67
NC_035780.1	8833124	8833126	60
NC_035780.1	12631453	12631455	60


In [17]:
!wc -l {hypoDML}

     288 ../../analyses/2018-10-25-MethylKit/2019-04-05-DML-Destrand-5x-Locations-Hypomethylated.bed


In [19]:
!head {hypoDML}

NC_035780.1	2538924	2538926	-50
NC_035780.1	2541726	2541728	-54
NC_035780.1	2586508	2586510	-53
NC_035780.1	4286802	4286804	-62
NC_035780.1	4288213	4288215	-58
NC_035780.1	4289628	4289630	-52
NC_035780.1	8693287	8693289	-52
NC_035780.1	9110274	9110276	-63
NC_035780.1	17093218	17093220	-52
NC_035780.1	17488958	17488960	-57


In [18]:
#Counting the number of DMR
!wc -l {DMRlist}

     162 ../../analyses/2018-10-25-MethylKit/2018-11-07-DMR-Locations.bed


In [17]:
!head {exonList}

NC_035780.1	13578	13603
NC_035780.1	14237	14290
NC_035780.1	14557	14594
NC_035780.1	28961	29073
NC_035780.1	30524	31557
NC_035780.1	31736	31887
NC_035780.1	31977	32565
NC_035780.1	32959	33324
NC_035780.1	43111	44358
NC_035780.1	43111	44358


In [18]:
!wc -l {exonList}

  731279 C_virginica-3.0_Gnomon_exon_sorted_yrv.bed


In [21]:
!head {intronList}

NC_035780.1	13603	14236
NC_035780.1	14290	14556
NC_035780.1	29073	30523
NC_035780.1	31557	31735
NC_035780.1	31887	31976
NC_035780.1	32565	32958
NC_035780.1	44358	45912
NC_035780.1	46506	64122
NC_035780.1	64334	66868
NC_035780.1	85777	88422


In [22]:
!wc -l {intronList}

  316614 C_virginica-3.0_Gnomon_intron_yrv.bed


In [24]:
!head {geneList}

NC_035780.1	13578	14594
NC_035780.1	28961	33324
NC_035780.1	43111	66897
NC_035780.1	85606	95254
NC_035780.1	99840	106460
NC_035780.1	108305	110077
NC_035780.1	151859	157536
NC_035780.1	163809	183798
NC_035780.1	164820	166793
NC_035780.1	169468	170178


In [25]:
!wc -l {geneList}

   38929 C_virginica-3.0_Gnomon_gene_sorted_yrv.bed


In [27]:
!head {transposableElementsAll}

##gff-version 2
##date 2018-08-23
##sequence-region Cvirginica_v300.fa
NC_007175.2	RepeatMasker	similarity	262	1389	31.1	+	.	Target "Motif:REP-6_LMi" 2920 4055
NC_007175.2	RepeatMasker	similarity	1728	1947	26.1	-	.	Target "Motif:REP-6_LMi" 14320 14534
NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	RepeatMasker	similarity	2129	2367	20.5	-	.	Target "Motif:REP-6_LMi" 13886 14118
NC_007175.2	RepeatMasker	similarity	2836	2980	31.5	+	.	Target "Motif:REP-6_LMi" 6216 6359
NC_007175.2	RepeatMasker	similarity	3196	3277	30.5	+	.	Target "Motif:REP-6_LMi" 6572 6653
NC_007175.2	RepeatMasker	similarity	5168	5532	32.9	+	.	Target "Motif:REP-6_LMi" 4620 4983


In [29]:
!wc -l {transposableElementsAll}

  692371 C_virginica-3,0_TE-all.gff


In [30]:
!head {transposableElementsCg}

##gff-version 2
##date 2018-08-27
##sequence-region Cvirginica_v300.fa
NC_007175.2	RepeatMasker	similarity	1866	2013	33.6	+	.	Target "Motif:LSU-rRNA_Cel" 2372 2520
NC_007175.2	RepeatMasker	similarity	6529	6628	19.0	+	.	Target "Motif:(TA)n" 2 102
NC_035780.1	RepeatMasker	similarity	1473	1535	 0.0	+	.	Target "Motif:(TAACCC)n" 1 63
NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	RepeatMasker	similarity	7423	7489	25.4	-	.	Target "Motif:Gypsy-62_CGi-I" 2097 2163
NC_035780.1	RepeatMasker	similarity	7623	8079	34.1	-	.	Target "Motif:Gypsy-62_CGi-I" 1516 1975
NC_035780.1	RepeatMasker	similarity	8261	8295	14.1	+	.	Target "Motif:(CTCCT)n" 1 33


In [31]:
!wc -l {transposableElementsCg}

  626665 C_virginica-3.0_TE-Cg.gff


In [25]:
!head {CGMotifList}

NC_035780.1	28	30	CG_motif
NC_035780.1	54	56	CG_motif
NC_035780.1	75	77	CG_motif
NC_035780.1	93	95	CG_motif
NC_035780.1	103	105	CG_motif
NC_035780.1	116	118	CG_motif
NC_035780.1	134	136	CG_motif
NC_035780.1	159	161	CG_motif
NC_035780.1	209	211	CG_motif
NC_035780.1	224	226	CG_motif


In [26]:
!wc -l {CGMotifList}

 14458703 C_virginica-3.0_CG-motif.bed


In [107]:
!head {mRNAList}

NC_035780.1	28961	33324
NC_035780.1	43111	66897
NC_035780.1	43111	46506
NC_035780.1	85606	95254
NC_035780.1	99840	106460
NC_035780.1	108305	110077
NC_035780.1	151859	157536
NC_035780.1	163809	183798
NC_035780.1	164820	166793
NC_035780.1	190449	193594


In [109]:
!wc -l {mRNAList}

   60201 C_virginica-3.0_Gnomon_mRNA_yrv.bed


## 2. Identify DML and DMR Overlaps with Genomic Feature Tracks

To identify the location of DML and DMR in the *C. virginica* genome, I will use `intersect` from `bedtools`. [The BEDtools suite](http://bedtools.readthedocs.io/en/latest/content/bedtools-suite.html) allows me to easily find overlapping regions of different bed files.

In [36]:
! {bedtoolsDirectory}intersectBed -h


Tool:    bedtools intersect (aka intersectBed)
Version: v2.26.0
Summary: Report overlaps between two feature files.

Usage:   bedtools intersect [OPTIONS] -a <bed/gff/vcf/bam> -b <bed/gff/vcf/bam>

	Note: -b may be followed with multiple databases and/or 
	wildcard (*) character(s). 
Options: 
	-wa	Write the original entry in A for each overlap.

	-wb	Write the original entry in B for each overlap.
		- Useful for knowing _what_ A overlaps. Restricted by -f and -r.

	-loj	Perform a "left outer join". That is, for each feature in A
		report each overlap with B.  If no overlaps are found, 
		report a NULL feature for B.

	-wo	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlaps restricted by -f and -r.
		  Only A features with overlap are reported.

	-wao	Write the original A and B entries plus the number of base
		pairs of overlap between the two features.
		- Overlapping features restricted by -f and -r.
		  However, A fea

### 2a. Exons

#### All DML

In [26]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {exonList} \
| wc -l
!echo "DML overlaps with exons"

     368
DML overlaps with exons


In [27]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {exonList} \
> 2019-05-29-DML-Exon.txt

In [28]:
!head 2019-05-29-DML-Exon.txt

NC_035780.1	571138	571140	58	NC_035780.1	570942	571194
NC_035780.1	2538924	2538926	-50	NC_035780.1	2538624	2538955
NC_035780.1	2586508	2586510	-53	NC_035780.1	2586438	2586557
NC_035780.1	2589720	2589722	57	NC_035780.1	2589716	2589955
NC_035780.1	4286286	4286288	67	NC_035780.1	4286174	4286407
NC_035780.1	4286802	4286804	-62	NC_035780.1	4286783	4286927
NC_035780.1	4289628	4289630	-52	NC_035780.1	4288592	4290756
NC_035780.1	8693287	8693289	-52	NC_035780.1	8692509	8693320
NC_035780.1	9110274	9110276	-63	NC_035780.1	9109982	9111843
NC_035780.1	12631453	12631455	60	NC_035780.1	12630576	12631487


#### Hypermethylated DML

In [29]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hyperDML} \
-b {exonList} \
| wc -l
!echo "hypermethylated DML overlaps with exons"

     190
hypermethylated DML overlaps with exons


In [30]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hyperDML} \
-b {exonList} \
> 2019-05-29-Hypermethylated-DML-Exon.txt

In [31]:
!head 2019-05-29-Hypermethylated-DML-Exon.txt

NC_035780.1	571138	571140	58	NC_035780.1	570942	571194
NC_035780.1	2589720	2589722	57	NC_035780.1	2589716	2589955
NC_035780.1	4286286	4286288	67	NC_035780.1	4286174	4286407
NC_035780.1	12631453	12631455	60	NC_035780.1	12630576	12631487
NC_035780.1	12631453	12631455	60	NC_035780.1	12630576	12631487
NC_035780.1	12631453	12631455	60	NC_035780.1	12630577	12631487
NC_035780.1	12631453	12631455	60	NC_035780.1	12630577	12631487
NC_035780.1	15412264	15412266	50	NC_035780.1	15412219	15412410
NC_035780.1	15412264	15412266	50	NC_035780.1	15412219	15412410
NC_035780.1	15414935	15414936	51	NC_035780.1	15414935	15415225


#### Hypomethylated DML

In [32]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hypoDML} \
-b {exonList} \
| wc -l
!echo "hypomethylated DML overlaps with exons"

     178
hypomethylated DML overlaps with exons


In [33]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hypoDML} \
-b {exonList} \
> 2019-05-29-Hypomethylated-DML-Exon.txt

In [34]:
!head 2019-05-29-Hypomethylated-DML-Exon.txt

NC_035780.1	2538924	2538926	-50	NC_035780.1	2538624	2538955
NC_035780.1	2586508	2586510	-53	NC_035780.1	2586438	2586557
NC_035780.1	4286802	4286804	-62	NC_035780.1	4286783	4286927
NC_035780.1	4289628	4289630	-52	NC_035780.1	4288592	4290756
NC_035780.1	8693287	8693289	-52	NC_035780.1	8692509	8693320
NC_035780.1	9110274	9110276	-63	NC_035780.1	9109982	9111843
NC_035780.1	17093218	17093220	-52	NC_035780.1	17092983	17093548
NC_035780.1	19149580	19149582	-61	NC_035780.1	19149513	19149749
NC_035780.1	19149580	19149582	-61	NC_035780.1	19149513	19149749
NC_035780.1	19149580	19149582	-61	NC_035780.1	19149513	19150486


#### DMR

In [35]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {exonList} \
| wc -l
!echo "DMR overlaps with exons"

/bin/sh: {bedtoolsDirectory}intersectBed: command not found
       0
DMR overlaps with exons


In [36]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {exonList} \
> 2018-11-07-DMR-Exon.txt

In [37]:
!head 2018-11-07-DMR-Exon.txt

NC_035780.1	571100	571194	DMR	58	NC_035780.1	570942	571194
NC_035780.1	573700	573800	DMR	52	NC_035780.1	573630	573906
NC_035780.1	573700	573800	DMR	52	NC_035780.1	573630	573906
NC_035780.1	1933574	1933600	DMR	53	NC_035780.1	1933574	1933615
NC_035780.1	47335000	47335100	DMR	-66	NC_035780.1	47334080	47336192
NC_035780.1	48394800	48394900	DMR	-63	NC_035780.1	48394159	48395287
NC_035780.1	61138200	61138300	DMR	-79	NC_035780.1	61138000	61140417
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302
NC_035781.1	6831300	6831302	DMR	-59	NC_035781.1	6831093	6831302


### 2b. Introns

#### DML

In [36]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {intronList} \
| wc -l
!echo "DML overlaps with introns"

     192
DML overlaps with introns


In [37]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {intronList} \
> 2019-05-29-DML-Intron.txt

In [38]:
!head 2019-05-29-DML-Intron.txt

NC_035780.1	401630	401632	53	NC_035780.1	401604	401800
NC_035780.1	1882691	1882693	64	NC_035780.1	1882355	1882971
NC_035780.1	1885022	1885024	61	NC_035780.1	1884754	1886042
NC_035780.1	1933499	1933501	51	NC_035780.1	1932876	1933573
NC_035780.1	2541726	2541728	-54	NC_035780.1	2538955	2541768
NC_035780.1	2584492	2584494	56	NC_035780.1	2584153	2584504
NC_035780.1	4288213	4288215	-58	NC_035780.1	4288128	4288230
NC_035780.1	8833124	8833126	60	NC_035780.1	8832171	8833699
NC_035780.1	17488958	17488960	-57	NC_035780.1	17488942	17489178
NC_035780.1	22177828	22177830	-51	NC_035780.1	22154686	22178240


#### Hypermethylated DML

In [39]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hyperDML} \
-b {intronList} \
| wc -l
!echo "hypermethylated DML overlaps with introns"

      99
hypermethylated DML overlaps with introns


In [40]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hyperDML} \
-b {intronList} \
> 2019-05-29-Hypermethylated-DML-Intron.txt

In [41]:
!head 2019-05-29-Hypermethylated-DML-Intron.txt

NC_035780.1	401630	401632	53	NC_035780.1	401604	401800
NC_035780.1	1882691	1882693	64	NC_035780.1	1882355	1882971
NC_035780.1	1885022	1885024	61	NC_035780.1	1884754	1886042
NC_035780.1	1933499	1933501	51	NC_035780.1	1932876	1933573
NC_035780.1	2584492	2584494	56	NC_035780.1	2584153	2584504
NC_035780.1	8833124	8833126	60	NC_035780.1	8832171	8833699
NC_035780.1	27396182	27396184	52	NC_035780.1	27396140	27396706
NC_035780.1	32766797	32766799	58	NC_035780.1	32766346	32769863
NC_035780.1	32766797	32766799	58	NC_035780.1	32766346	32769863
NC_035780.1	38236493	38236495	50	NC_035780.1	38236121	38236506


#### Hypomethylated DML

In [42]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hypoDML} \
-b {intronList} \
| wc -l
!echo "hypomethylated DML overlaps with introns"

      93
hypomethylated DML overlaps with introns


In [43]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hypoDML} \
-b {intronList} \
> 2019-05-29-Hypomethylated-DML-Intron.txt

In [44]:
!head 2019-05-29-Hypomethylated-DML-Intron.txt

NC_035780.1	2541726	2541728	-54	NC_035780.1	2538955	2541768
NC_035780.1	4288213	4288215	-58	NC_035780.1	4288128	4288230
NC_035780.1	17488958	17488960	-57	NC_035780.1	17488942	17489178
NC_035780.1	22177828	22177830	-51	NC_035780.1	22154686	22178240
NC_035780.1	22177828	22177830	-51	NC_035780.1	22154686	22178240
NC_035780.1	25858297	25858299	-51	NC_035780.1	25858281	25863048
NC_035780.1	31302904	31302906	-60	NC_035780.1	31302841	31303151
NC_035780.1	31302934	31302936	-58	NC_035780.1	31302841	31303151
NC_035780.1	32717030	32717032	-52	NC_035780.1	32716795	32717179
NC_035780.1	35969128	35969130	-53	NC_035780.1	35969070	35986498


#### DMR

In [41]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {intronList} \
| wc -l
!echo "DMR overlaps with introns"

     112
DMR overlaps with introns


In [42]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {intronList} \
> 2018-11-07-DMR-Intron.txt

In [43]:
!head 2018-11-07-DMR-Intron.txt

NC_035780.1	571195	571200	DMR	58	NC_035780.1	571195	572677
NC_035780.1	1885000	1885100	DMR	51	NC_035780.1	1884755	1886043
NC_035780.1	1933500	1933574	DMR	53	NC_035780.1	1932877	1933574
NC_035780.1	4285700	4285800	DMR	-51	NC_035780.1	4285382	4285831
NC_035780.1	26629500	26629600	DMR	65	NC_035780.1	26621919	26632637
NC_035780.1	28563400	28563500	DMR	59	NC_035780.1	28563400	28564616
NC_035780.1	29883000	29883100	DMR	-55	NC_035780.1	29882984	29883643
NC_035780.1	31302900	31303000	DMR	-61	NC_035780.1	31302842	31303152
NC_035780.1	31303300	31303400	DMR	-53	NC_035780.1	31303293	31303603
NC_035780.1	33209900	33210000	DMR	-54	NC_035780.1	33209785	33210978


### 2c. Genes

#### DML

In [45]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {geneList} \
| wc -l
!echo "DML overlaps with genes"

     560
DML overlaps with genes


In [55]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {geneList} \
> 2019-05-29-DML-Genes.txt

In [56]:
!head 2019-05-29-DML-Genes.txt

NC_035780.1	401630	401632	53	NC_035780.1	394983	409280
NC_035780.1	571138	571140	58	NC_035780.1	544088	573497
NC_035780.1	1882691	1882693	64	NC_035780.1	1882143	1890106
NC_035780.1	1885022	1885024	61	NC_035780.1	1882143	1890106
NC_035780.1	1933499	1933501	51	NC_035780.1	1928718	1940217
NC_035780.1	2538924	2538926	-50	NC_035780.1	2524425	2553408
NC_035780.1	2541726	2541728	-54	NC_035780.1	2524425	2553408
NC_035780.1	2584492	2584494	56	NC_035780.1	2554181	2599559
NC_035780.1	2586508	2586510	-53	NC_035780.1	2554181	2599559
NC_035780.1	2589720	2589722	57	NC_035780.1	2554181	2599559


I know how many overlaps there are, but I also want to know how many unique genes have DMLs in them. For this, I will use the following code:

`cut -f7 2019-05-29-DML-Genes.txt | sort | uniq -c`

`cut` is the command that isolates the column information. Each gene has a unique end position, so I'll look at unique entries in the seventh column (`-f7`). The column is piped into `sort`, then that output is counted for unique lines by `uniq`. Finally, I'll pipe this into `wc -l` to count the number of unique genes.

In [57]:
! cut -f7 2019-05-29-DML-Genes.txt | sort | uniq -c | wc -l
!echo "unique genes overlapping with DML"

     481
unique genes overlapping with DML


#### Hypermethylated DML

In [58]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hyperDML} \
-b {geneList} \
| wc -l
!echo "hypermethylated DML overlaps with genes"

     289
hypermethylated DML overlaps with genes


In [59]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hyperDML} \
-b {geneList} \
> 2019-05-29-Hypermethylated-DML-Genes.txt

In [60]:
!head 2019-05-29-Hypermethylated-DML-Genes.txt

NC_035780.1	401630	401632	53	NC_035780.1	394983	409280
NC_035780.1	571138	571140	58	NC_035780.1	544088	573497
NC_035780.1	1882691	1882693	64	NC_035780.1	1882143	1890106
NC_035780.1	1885022	1885024	61	NC_035780.1	1882143	1890106
NC_035780.1	1933499	1933501	51	NC_035780.1	1928718	1940217
NC_035780.1	2584492	2584494	56	NC_035780.1	2554181	2599559
NC_035780.1	2589720	2589722	57	NC_035780.1	2554181	2599559
NC_035780.1	4286286	4286288	67	NC_035780.1	4282771	4298209
NC_035780.1	8833124	8833126	60	NC_035780.1	8829533	8833841
NC_035780.1	12631453	12631455	60	NC_035780.1	12630576	12697104


In [63]:
! cut -f7 2019-05-29-Hypermethylated-DML-Genes.txt | sort | uniq -c | wc -l
!echo "unique genes overlapping with hypermethylated DML"

     269
unique genes overlapping with hypermethylated DML


#### Hypomethylated DML

In [64]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hypoDML} \
-b {geneList} \
| wc -l
!echo "hypomethylated DML overlaps with genes"

     271
hypomethylated DML overlaps with genes


In [65]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hypoDML} \
-b {geneList} \
> 2019-05-29-Hypomethylated-DML-mRNA.txt

In [66]:
!head 2019-05-29-Hypomethylated-DML-mRNA.txt

NC_035780.1	2538924	2538926	-50	NC_035780.1	2524425	2553408
NC_035780.1	2541726	2541728	-54	NC_035780.1	2524425	2553408
NC_035780.1	2586508	2586510	-53	NC_035780.1	2554181	2599559
NC_035780.1	4286802	4286804	-62	NC_035780.1	4282771	4298209
NC_035780.1	4288213	4288215	-58	NC_035780.1	4282771	4298209
NC_035780.1	4289628	4289630	-52	NC_035780.1	4282771	4298209
NC_035780.1	8693287	8693289	-52	NC_035780.1	8692509	8698183
NC_035780.1	9110274	9110276	-63	NC_035780.1	9103662	9111843
NC_035780.1	17093218	17093220	-52	NC_035780.1	17089706	17093548
NC_035780.1	17488958	17488960	-57	NC_035780.1	17457431	17541765


In [67]:
! cut -f7 2019-05-29-Hypomethylated-DML-mRNA.txt | sort | uniq -c | wc -l
!echo "unique genes overlapping with hypomethylated DML"

     241
unique genes overlapping with hypomethylated DML


#### DMR

In [50]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {mRNAList} \
| wc -l
!echo "DMR overlaps with mRNA"

     139
DMR overlaps with mRNA


In [51]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {mRNAList} \
> 2018-11-07-DMR-mRNA.txt

In [52]:
!head 2018-11-07-DMR-mRNA.txt

NC_035780.1	571100	571200	DMR	58	NC_035780.1	Gnomon	mRNA	544088	573497	.	+	.	ID=rna48;Parent=gene35;Dbxref=GeneID:111114201,Genbank:XM_022452489.1;Name=XM_022452489.1;Note=The sequence of the model RefSeq transcript was modified relative to this genomic sequence to represent the inferred CDS: inserted 2 bases in 2 codons;exception=unclassified transcription discrepancy;gbkey=mRNA;gene=LOC111114201;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 99%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 9 samples with support for all annotated introns;product=vacuolar protein sorting-associated protein 13B-like;transcript_id=XM_022452489.1
NC_035780.1	573700	573800	DMR	52	NC_035780.1	Gnomon	mRNA	573630	585444	.	-	.	ID=rna49;Parent=gene36;Dbxref=GeneID:111114212,Genbank:XM_022452506.1;Name=XM_022452506.1;gbkey=mRNA;gene=LOC111114212;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 100%25 coverage of the annot

In [53]:
! cut -f14 2018-11-07-DMR-mRNA.txt | sort | uniq -c > 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

In [54]:
!head 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

   1 ID=rna10182;Parent=gene5989;Dbxref=GeneID:111121400,Genbank:XM_022462674.1;Name=XM_022462674.1;gbkey=mRNA;gene=LOC111121400;model_evidence=Supporting evidence includes similarity to: 3 Proteins%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 16 samples with support for all annotated introns;product=DNA excision repair protein ERCC-6-like;transcript_id=XM_022462674.1
   1 ID=rna10216;Parent=gene6005;Dbxref=GeneID:111120829,Genbank:XM_022461817.1;Name=XM_022461817.1;gbkey=mRNA;gene=LOC111120829;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 23 samples with support for all annotated introns;product=serine/arginine-rich splicing factor 7-like%2C transcript variant X1;transcript_id=XM_022461817.1
   1 ID=rna10452;Parent=gene6155;Dbxref=GeneID:111118143,Genbank:XM_022457464.1;Name=XM_022457464.1;Note=The sequence of the model RefSeq tr

In [55]:
!wc -l 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt

     305 2018-11-07-Unique-Genes-in-DMR-mRNA-Overlap.txt


The DMRs overlap with 305 unique genes.

### 2c. Transposable Elements (All)

#### DML

In [68]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {transposableElementsAll} \
| wc -l
!echo "DML overlaps with transposable elements (all)"

      57
DML overlaps with transposable elements (all)


In [69]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {transposableElementsAll} \
> 2019-05-29-DML-TE-all.txt

In [70]:
!head 2019-05-29-DML-TE-all.txt

NC_035780.1	8833124	8833126	60	NC_035780.1	RepeatMasker	similarity	8833042	8833288	18.2	-	.	Target "Motif:CVA" 1 272
NC_035780.1	22177828	22177830	-51	NC_035780.1	RepeatMasker	similarity	22177766	22177877	22.3	-	.	Target "Motif:DNA9-6_CGi" 1 115
NC_035780.1	57337100	57337102	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035780.1	58135767	58135769	74	NC_035780.1	RepeatMasker	similarity	58135699	58135837	22.4	+	.	Target "Motif:BivaMD-SINE1_CrVi" 169 314
NC_035781.1	22439769	22439771	53	NC_035781.1	RepeatMasker	similarity	22439740	22439796	28.1	+	.	Target "Motif:Mariner-6_AMi" 698 754
NC_035781.1	29178318	29178320	-55	NC_035781.1	RepeatMasker	similarity	29177336	29178341	16.0	-	.	Target "Motif:CVA" 2 863
NC_035781.1	54151548	54151550	54	NC_035781.1	RepeatMasker	similarity	54150482	54151750	14.3	+	.	Target "Motif:CVA" 1 1018
NC_035781.1	59742649	59742651	-65	NC_035781.1	RepeatMasker	similarity	59742603	59742651	 4.2	+	.	Target "Moti

#### Hypermethylated DML

In [71]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hyperDML} \
-b {transposableElementsAll} \
| wc -l
!echo "hypermethylated DML overlaps with TE (all)"

      26
hypermethylated DML overlaps with TE (all)


In [72]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hyperDML} \
-b {transposableElementsAll} \
> 2019-05-29-Hypermethylated-DML-TEall.txt

In [73]:
!head 2019-05-29-Hypermethylated-DML-TEall.txt

NC_035780.1	8833124	8833126	60	NC_035780.1	RepeatMasker	similarity	8833042	8833288	18.2	-	.	Target "Motif:CVA" 1 272
NC_035780.1	58135767	58135769	74	NC_035780.1	RepeatMasker	similarity	58135699	58135837	22.4	+	.	Target "Motif:BivaMD-SINE1_CrVi" 169 314
NC_035781.1	22439769	22439771	53	NC_035781.1	RepeatMasker	similarity	22439740	22439796	28.1	+	.	Target "Motif:Mariner-6_AMi" 698 754
NC_035781.1	54151548	54151550	54	NC_035781.1	RepeatMasker	similarity	54150482	54151750	14.3	+	.	Target "Motif:CVA" 1 1018
NC_035782.1	45857195	45857197	52	NC_035782.1	RepeatMasker	similarity	45857026	45858123	32.7	+	.	Target "Motif:Mariner-21_LCh" 450 1999
NC_035782.1	53693367	53693369	61	NC_035782.1	RepeatMasker	similarity	53693299	53693466	19.6	+	.	Target "Motif:Crypton-N6B_CGi" 566 735
NC_035782.1	58675269	58675271	50	NC_035782.1	RepeatMasker	similarity	58675249	58675337	19.1	-	.	Target "Motif:DNA3-12_CGi" 290 378
NC_035782.1	61203970	61203972	51	NC_035782.1	RepeatMasker	similarity	61203541	61204351	12.

#### Hypomethylated DML

In [36]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hypoDML} \
-b {transposableElementsAll} \
| wc -l
!echo "hypomethylated DML overlaps with TE (all)"

      31
hypomethylated DML overlaps with TE (all)


In [74]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hypoDML} \
-b {transposableElementsAll} \
> 2019-05-29-Hypomethylated-DML-TEall.txt

In [75]:
!head 2019-05-29-Hypomethylated-DML-TEall.txt

NC_035780.1	22177828	22177830	-51	NC_035780.1	RepeatMasker	similarity	22177766	22177877	22.3	-	.	Target "Motif:DNA9-6_CGi" 1 115
NC_035780.1	57337100	57337102	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035781.1	29178318	29178320	-55	NC_035781.1	RepeatMasker	similarity	29177336	29178341	16.0	-	.	Target "Motif:CVA" 2 863
NC_035781.1	59742649	59742651	-65	NC_035781.1	RepeatMasker	similarity	59742603	59742651	 4.2	+	.	Target "Motif:(ACTAACG)n" 1 49
NC_035782.1	6685343	6685345	-68	NC_035782.1	RepeatMasker	similarity	6685308	6685646	15.0	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 335
NC_035782.1	6685349	6685351	-50	NC_035782.1	RepeatMasker	similarity	6685308	6685646	15.0	+	.	Target "Motif:BivaMD-SINE1_CrVi" 1 335
NC_035782.1	34498893	34498895	-55	NC_035782.1	RepeatMasker	similarity	34498501	34500091	24.8	+	.	Target "Motif:Helitron-N40_CGi" 1 1569
NC_035782.1	34498895	34498897	-71	NC_035782.1	RepeatMasker	similarity	34498501	34500091	24

#### DMR

In [59]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsAll} \
| wc -l
!echo "DMR overlaps with transposable elements (all)"

      39
DMR overlaps with transposable elements (all)


In [138]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsAll} \
> 2018-11-07-DMR-TE-all.txt

In [139]:
!head 2018-11-07-DMR-TE-all.txt

NC_035780.1	24159600	24159700	DMR	51
NC_035780.1	29883000	29883100	DMR	-55
NC_035780.1	33945900	33946000	DMR	-51
NC_035780.1	46044700	46044800	DMR	-53
NC_035780.1	47335000	47335100	DMR	-66
NC_035781.1	30962200	30962300	DMR	63
NC_035781.1	51566900	51567000	DMR	-61
NC_035781.1	54151500	54151600	DMR	55
NC_035782.1	2787300	2787400	DMR	-53
NC_035782.1	7518400	7518500	DMR	64


### 2e. Transposable Elements (_C. gigas_ only)

#### DML

In [29]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMLlist} \
-b {transposableElementsCg} \
| wc -l
!echo "DML overlaps with transposable elements (Cg)"

      39
DML overlaps with transposable elements (Cg)


In [76]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMLlist} \
-b {transposableElementsCg} \
> 2019-05-29-DML-TE-Cg.txt

In [77]:
!head 2019-05-29-DML-TE-Cg.txt

NC_035780.1	8833124	8833126	60	NC_035780.1	RepeatMasker	similarity	8833045	8833287	22.6	-	.	Target "Motif:Helitron-N2f_CGi" 1 276
NC_035780.1	22177828	22177830	-51	NC_035780.1	RepeatMasker	similarity	22177766	22177877	22.3	-	.	Target "Motif:DNA9-6_CGi" 1 115
NC_035780.1	57337100	57337102	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035781.1	29178318	29178320	-55	NC_035781.1	RepeatMasker	similarity	29177333	29178341	24.4	-	.	Target "Motif:Helitron-N2d_CGi" 2 863
NC_035781.1	54151548	54151550	54	NC_035781.1	RepeatMasker	similarity	54150483	54151741	23.3	+	.	Target "Motif:Helitron-N2f_CGi" 1 1018
NC_035781.1	59742649	59742651	-65	NC_035781.1	RepeatMasker	similarity	59742603	59742651	 4.2	+	.	Target "Motif:(ACTAACG)n" 1 49
NC_035782.1	34498893	34498895	-55	NC_035782.1	RepeatMasker	similarity	34498501	34500091	24.8	+	.	Target "Motif:Helitron-N40_CGi" 1 1569
NC_035782.1	34498895	34498897	-71	NC_035782.1	RepeatMasker	similarity	344985

#### Hypermethylated DML

In [32]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {hyperDML} \
-b {transposableElementsCg} \
| wc -l
!echo "hypermethylated DML overlaps with TE (Cg)"

      16
hypermethylated DML overlaps with TE (Cg)


In [80]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hyperDML} \
-b {transposableElementsCg} \
> 2019-05-29-Hypermethylated-DML-TECg.txt

In [81]:
!head 2019-05-29-Hypermethylated-DML-TECg.txt

NC_035780.1	8833124	8833126	60	NC_035780.1	RepeatMasker	similarity	8833045	8833287	22.6	-	.	Target "Motif:Helitron-N2f_CGi" 1 276
NC_035781.1	54151548	54151550	54	NC_035781.1	RepeatMasker	similarity	54150483	54151741	23.3	+	.	Target "Motif:Helitron-N2f_CGi" 1 1018
NC_035782.1	53693367	53693369	61	NC_035782.1	RepeatMasker	similarity	53693299	53693466	19.6	+	.	Target "Motif:Crypton-N6B_CGi" 566 735
NC_035782.1	58675269	58675271	50	NC_035782.1	RepeatMasker	similarity	58675249	58675337	19.1	-	.	Target "Motif:DNA3-12_CGi" 290 378
NC_035782.1	61203970	61203972	51	NC_035782.1	RepeatMasker	similarity	61203650	61204350	24.8	-	.	Target "Motif:Helitron-N2d_CGi" 1 686
NC_035783.1	4336100	4336102	63	NC_035783.1	RepeatMasker	similarity	4335884	4336135	21.8	+	.	Target "Motif:DNA8-4_CGi" 42 268
NC_035783.1	23130125	23130127	53	NC_035783.1	RepeatMasker	similarity	23130086	23130209	18.6	+	.	Target "Motif:Crypton-8N1_CGi" 516 639
NC_035783.1	29749414	29749416	57	NC_035783.1	RepeatMasker	similarity	297493

#### Hypomethylated DML

In [35]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {hypoDML} \
-b {transposableElementsCg} \
| wc -l
!echo "hypomethylated DML overlaps with TE (Cg)"

      23
hypomethylated DML overlaps with TE (Cg)


In [82]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {hypoDML} \
-b {transposableElementsCg} \
> 2019-05-29-Hypomethylated-DML-TECg.txt

In [83]:
!head 2019-05-29-Hypomethylated-DML-TECg.txt

NC_035780.1	22177828	22177830	-51	NC_035780.1	RepeatMasker	similarity	22177766	22177877	22.3	-	.	Target "Motif:DNA9-6_CGi" 1 115
NC_035780.1	57337100	57337102	-54	NC_035780.1	RepeatMasker	similarity	57337042	57337128	18.6	-	.	Target "Motif:DNA2-2_CGi" 413 498
NC_035781.1	29178318	29178320	-55	NC_035781.1	RepeatMasker	similarity	29177333	29178341	24.4	-	.	Target "Motif:Helitron-N2d_CGi" 2 863
NC_035781.1	59742649	59742651	-65	NC_035781.1	RepeatMasker	similarity	59742603	59742651	 4.2	+	.	Target "Motif:(ACTAACG)n" 1 49
NC_035782.1	34498893	34498895	-55	NC_035782.1	RepeatMasker	similarity	34498501	34500091	24.8	+	.	Target "Motif:Helitron-N40_CGi" 1 1569
NC_035782.1	34498895	34498897	-71	NC_035782.1	RepeatMasker	similarity	34498501	34500091	24.8	+	.	Target "Motif:Helitron-N40_CGi" 1 1569
NC_035783.1	48434286	48434288	-53	NC_035783.1	RepeatMasker	similarity	48434172	48434360	26.1	-	.	Target "Motif:DNA3-11_CGi" 1856 2040
NC_035783.1	49079096	49079097	-50	NC_035783.1	RepeatMasker	similarity	4

#### DMR

In [22]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {DMRlist} \
-b {transposableElementsCg} \
| wc -l
!echo "DMR overlaps with transposable elements (Cg)"

      23
DMR overlaps with transposable elements (Cg)


In [66]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {DMRlist} \
-b {transposableElementsCg} \
> 2018-11-07-DMR-TE-Cg.txt

In [67]:
!head 2018-11-07-DMR-TE-Cg.txt

NC_035780.1	29883075	29883100	DMR	-55	NC_035780.1	RepeatMasker	similarity	29883076	29883262	25.4	+	.	Target "Motif:DNA9-4_CGi" 776 964
NC_035780.1	33945935	33945973	DMR	-51	NC_035780.1	RepeatMasker	similarity	33945936	33945973	 5.3	+	.	Target "Motif:DNA9-5_CGi" 1 38
NC_035780.1	33945972	33946000	DMR	-51	NC_035780.1	RepeatMasker	similarity	33945973	33946026	 3.7	+	.	Target "Motif:DNA8-12_CGi" 645 699
NC_035781.1	30962200	30962300	DMR	63	NC_035781.1	RepeatMasker	similarity	30962179	30962531	18.4	+	.	Target "Motif:Helitron-N2d_CGi" 3 346
NC_035781.1	51566900	51566921	DMR	-61	NC_035781.1	RepeatMasker	similarity	51566285	51566921	25.2	-	.	Target "Motif:Sola3-1_CGi" 3273 3922
NC_035781.1	51566900	51567000	DMR	-61	NC_035781.1	RepeatMasker	similarity	51566310	51567155	27.8	-	.	Target "Motif:Helitron-N43_CGi" 1 1221
NC_035781.1	54151500	54151600	DMR	55	NC_035781.1	RepeatMasker	similarity	54150483	54151741	23.3	+	.	Target "Motif:Helitron-N2f_CGi" 1 1018
NC_035782.1	2787300	2787318	DMR	-53	NC_035

## 3. Identify Overlaps between CG Motif and Other Genome Feature Tracks

I began some of this work in [this Jupyter notebook](https://github.com/fish546-2018/yaamini-virginica/blob/master/notebooks/2019-05-13-Generating-Genome-Feature-Tracks.ipynb). Now I'll continue this for transposable elements.

## 3. Identify Overlaps between Transposable Elements and Other Genome Feature Tracks

To fully understand my results, I also need to know where TEs are located with respect to exons, introns, and mRNA coding regions.

### 3a. Transposable Elements (All)

#### Exons

In [85]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {transposableElementsAll} \
| wc -l
!echo "Exon overlaps with transposable elements (all)"

   50331
Exon overlaps with transposable elements (all)


In [102]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {transposableElementsAll} \
> 2018-11-07-Exon-TE-all.txt

In [103]:
!head 2018-11-07-Exon-TE-all.txt

NC_035780.1	109967	109996	NC_035780.1	RepeatMasker	similarity	109968	109996	 0.0	+	.	Target "Motif:(CCT)n" 1 29
NC_035780.1	164885	164914	NC_035780.1	RepeatMasker	similarity	164886	164914	 7.3	+	.	Target "Motif:(GAG)n" 1 29
NC_035780.1	166074	166280	NC_035780.1	RepeatMasker	similarity	166075	166280	32.8	+	.	Target "Motif:Harbinger1_DR" 1472 1676
NC_035780.1	166500	166566	NC_035780.1	RepeatMasker	similarity	166501	166566	30.3	+	.	Target "Motif:Harbinger-6_DR" 1152 1217
NC_035780.1	166597	166642	NC_035780.1	RepeatMasker	similarity	166598	166642	17.8	+	.	Target "Motif:hATw-1_HM" 2778 2822
NC_035780.1	220121	220199	NC_035780.1	RepeatMasker	similarity	220122	220199	24.7	-	.	Target "Motif:Gypsy-75_CQ-I" 1012 1091
NC_035780.1	228341	228392	NC_035780.1	RepeatMasker	similarity	228342	228392	20.0	+	.	Target "Motif:RTE-3_Hmel" 1405 1455
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	228341	228392	NC_035780.1	RepeatMasker

#### Introns

In [86]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {transposableElementsAll} \
| wc -l
!echo "Intron overlaps with transposable elements (all)"

  115151
Intron overlaps with transposable elements (all)


In [88]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {transposableElementsAll} \
> 2018-11-07-Intron-TE-all.txt

In [89]:
!head 2018-11-07-Intron-TE-all.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	48462	48520	NC_035780.1	RepeatMasker	similarity	48463	48520	 8.8	+	.	Target "Motif:BivaMD-SINE1_CrVi" 280 337
NC_035780.1	48665	49000	NC_035780.1	RepeatMasker	similarity	48666	49000	10.9	-	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	53155	53294	NC_035780.1	RepeatMasker	similarity	53156	532

#### Genes

In [92]:
!{bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {transposableElementsAll} \
| wc -l
!echo "gene overlaps with transposable elements (all)"

   33739
gene overlaps with transposable elements (all)


In [95]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {geneList} \
-b {transposableElementsAll} \
> 2018-11-07-Genes-TE-all.txt

In [96]:
!head 2018-11-07-Genes-TE-all.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	48462	48520	NC_035780.1	RepeatMasker	similarity	48463	48520	 8.8	+	.	Target "Motif:BivaMD-SINE1_CrVi" 280 337
NC_035780.1	48665	49000	NC_035780.1	RepeatMasker	similarity	48666	49000	10.9	-	.	Target "Motif:BivaMD-SINE1_CrVi" 1 337
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	53155	53294	NC_035780.1	RepeatMasker	similarity	53156	532

#### CG motifs

In [48]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {CGMotifList} \
-b {transposableElementsAll} \
| wc -l
!echo "CG motif overlaps with transposable elements (all)"

 2828372
CG motif overlaps with transposable elements (all)


In [49]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {CGMotifList} \
-b {transposableElementsAll} \
> 2018-11-07-TE-all-CGmotif.txt

In [50]:
!head 2018-11-07-TE-all-CGmotif.txt

NC_035780.1	5079	5080	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5159	5161	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5162	5164	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5174	5176	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5191	5193	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5220	5222	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5317	5319	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5357	5359	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 21

### 4b. Transposable Elements (_C. gigas_ only)

#### Exons

In [24]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {exonList} \
-b {transposableElementsCg} \
| wc -l
!echo "Exon overlaps with transposable elements (Cg)"

   41511
Exon overlaps with transposable elements (Cg)


In [27]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {exonList} \
-b {transposableElementsCg} \
> 2018-11-07-Exon-TE-Cg.txt

In [28]:
!head 2018-11-07-Exon-TE-Cg.txt

NC_035780.1	109967	109996	NC_035780.1	RepeatMasker	similarity	109968	109996	 0.0	+	.	Target "Motif:(CCT)n" 1 29
NC_035780.1	164885	164914	NC_035780.1	RepeatMasker	similarity	164886	164914	 7.3	+	.	Target "Motif:(GAG)n" 1 29
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	227767	227819	NC_035780.1	RepeatMasker	similarity	227768	227819	25.0	+	.	Target "Motif:A-rich" 1 54
NC_035780.1	233475	233478	NC_035780.1	RepeatMasker	similarity	233445	233478	10.1	+	.	Target "Motif:(CCTTT)n" 1 35
NC_035780.1	232863	233028	NC_035780.1	RepeatMasker	similarity	232798	233028	29.7	-	.	Target "Motif:ISL2EU-N8_CGi" 15 237
NC_035780.1	269562	269603	NC_035780.1	RepeatMasker	similarity	269563	269603	17.1	+	.	Target "Motif:(ATG)n" 1 42
NC_035780.1	258539	258574	NC_035780.1	RepeatMasker	similarity	258540	258574	16.3	+	.	Target "

#### Introns

In [100]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {intronList} \
-b {transposableElementsCg} \
| wc -l
!echo "Intron overlaps with transposable elements (Cg)"

  107542
Intron overlaps with transposable elements (Cg)


In [101]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {intronList} \
-b {transposableElementsCg} \
> 2018-11-07-Intron-TE-Cg.txt

In [102]:
!head 2018-11-07-Intron-TE-Cg.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	46753	46805	NC_035780.1	RepeatMasker	similarity	46754	46805	 6.8	+	.	Target "Motif:DNA-22_CGi" 631 722
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	86839	86942	NC_035780.1	RepeatMasker	similarity	86840	86942	27.4	-	.	Target "Motif:Helitron-N14_CGi" 83 189
NC_035780.1	87408	87513	NC_035780.1	RepeatMasker	similarity	87409	87513	19.8

#### Genes

In [97]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {geneList} \
-b {transposableElementsCg} \
| wc -l
!echo "gene overlaps with transposable elements (Cg)"

   32705
gene overlaps with transposable elements (Cg)


In [98]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {geneList} \
-b {transposableElementsCg} \
> 2018-11-07-Gene-TE-Cg.txt

In [99]:
!head 2018-11-07-Gene-TE-Cg.txt

NC_035780.1	32719	32819	NC_035780.1	RepeatMasker	similarity	32720	32819	18.2	+	.	Target "Motif:Crypton-9N1_CGi" 239 337
NC_035780.1	46753	46805	NC_035780.1	RepeatMasker	similarity	46754	46805	 6.8	+	.	Target "Motif:DNA-22_CGi" 631 722
NC_035780.1	50250	50279	NC_035780.1	RepeatMasker	similarity	50251	50279	 0.0	+	.	Target "Motif:(GGTTAG)n" 1 29
NC_035780.1	50605	50760	NC_035780.1	RepeatMasker	similarity	50606	50760	21.3	+	.	Target "Motif:Harbinger-2N1_CGi" 1 166
NC_035780.1	50976	51034	NC_035780.1	RepeatMasker	similarity	50977	51034	 0.0	+	.	Target "Motif:(TA)n" 1 58
NC_035780.1	51455	51498	NC_035780.1	RepeatMasker	similarity	51456	51498	 0.0	+	.	Target "Motif:(AG)n" 1 43
NC_035780.1	51720	51922	NC_035780.1	RepeatMasker	similarity	51721	51922	21.8	+	.	Target "Motif:Harbinger-2N1_CGi" 2568 2776
NC_035780.1	86839	86942	NC_035780.1	RepeatMasker	similarity	86840	86942	27.4	-	.	Target "Motif:Helitron-N14_CGi" 83 189
NC_035780.1	87408	87513	NC_035780.1	RepeatMasker	similarity	87409	87513	19.8

#### CG motifs

In [84]:
! {bedtoolsDirectory}intersectBed \
-u \
-a {CGMotifList} \
-b {transposableElementsCg} \
| wc -l
!echo "CG motif overlaps with transposable elements (Cg)"

 2142774
CG motif overlaps with transposable elements (Cg)


In [52]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a {CGMotifList} \
-b {transposableElementsCg} \
> 2018-11-07-TE-Cg-CGmotif.txt

In [53]:
!head 2018-11-07-TE-Cg-CGmotif.txt

NC_035780.1	5079	5080	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5159	5161	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5162	5164	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5174	5176	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5191	5193	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5220	5222	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5317	5319	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 2102 4631
NC_035780.1	5357	5359	CG_motif	NC_035780.1	RepeatMasker	similarity	5080	7289	32.5	-	.	Target "Motif:Gypsy-62_CGi-I" 21

## 4. Gene Flanking

I will perform a flanking analysis in two ways. First, I will use `bedtools flank` to add 1000 bp regions to each mRNA coding region. I can then isolate these flanks and intersect them with various genomic feature files. Second I will use `bedtools closest` to find the closest non-overlapping DML or DMR to each mRNA coding region.

In [110]:
mkdir 2019-05-29-Flanking-Analysis #Create a new directory for flanking analysis output

### 6a. `flank`

I also need to know if DMLs and CG motifs overlap with regions that flank mRNA. These flanking regions could be promoters or transcription factors that could regulate these processes. To do this, I will use `bedtools flank`:

1. Path to `flankBed`
2. -i: Path to mRNA GFF file
3. -g: Path to C. virginica "genome" file. flankBed requires the start and stop position of each genome (see this issue). I created a file like in TextWrangler using chromosome lengths from NCBI.
4. -b 1000: Add 1000 bp flanks to each end of the coding region

In [111]:
! {bedtoolsDirectory}flankBed \
-i {mRNAList} \
-g 2018-11-14-Flanking-Analysis/2018-11-14-bedtools-Chromosome-Length.txt \
-b 1000 \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-1000bp-Flanks.bed

In [112]:
!head {mRNAList} #The original file, just for comparison

NC_035780.1	28961	33324
NC_035780.1	43111	66897
NC_035780.1	43111	46506
NC_035780.1	85606	95254
NC_035780.1	99840	106460
NC_035780.1	108305	110077
NC_035780.1	151859	157536
NC_035780.1	163809	183798
NC_035780.1	164820	166793
NC_035780.1	190449	193594


In [113]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-1000bp-Flanks.bed #Isolated flanks. The first entry is the upstream flank for the first mRNA coding region, second is the downstream flank for the mRNA coding region, etc.

NC_035780.1	27961	28961
NC_035780.1	33324	34324
NC_035780.1	42111	43111
NC_035780.1	66897	67897
NC_035780.1	42111	43111
NC_035780.1	46506	47506
NC_035780.1	84606	85606
NC_035780.1	95254	96254
NC_035780.1	98840	99840
NC_035780.1	106460	107460


In [114]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-1000bp-Flanks.bed

  120402 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-1000bp-Flanks.bed


Now that I have these flanks, I want to separate the upstream flank from the downstream flank. I will do this using `awk`. If th row number is odd, the rows go into the upstream flank file. If the row number is even, it goes into the downstream flank file.

In [115]:
!awk '{ if (NR%2) print > "2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed"; \
else print > "2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed" }' \
2019-05-29-Flanking-Analysis/2019-05-29-mRNA-1000bp-Flanks.bed

#### Upstream flank

In [116]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed

NC_035780.1	27961	28961
NC_035780.1	42111	43111
NC_035780.1	42111	43111
NC_035780.1	84606	85606
NC_035780.1	98840	99840
NC_035780.1	107305	108305
NC_035780.1	150859	151859
NC_035780.1	162809	163809
NC_035780.1	163820	164820
NC_035780.1	189449	190449


In [118]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed

   60201 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed


There are just as many upstream flanks as there are mRNA, so that's good!

#### Downstream flanks

In [120]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed

NC_035780.1	33324	34324
NC_035780.1	66897	67897
NC_035780.1	46506	47506
NC_035780.1	95254	96254
NC_035780.1	106460	107460
NC_035780.1	110077	111077
NC_035780.1	157536	158536
NC_035780.1	183798	184798
NC_035780.1	166793	167793
NC_035780.1	193594	194594


In [121]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed

   60201 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed


Now I'll take the upstream and downstream flank BEDfiles I made and use it in intersectBed to find overlaps with DML, DMR, and CG motifs!

1. Path to intersectBed
2. -wb: Write output according to the second file
3. -a: Path to BEDfile created with flanks
4. -b: Specify either DML, DMR, or CG motif file. Overlaps between the flanks and CG motifs can be used as a background when comparing DML-flank and DMR-flank results
5. ">" filename: Redirect output to a .txt file

#### DML

In [122]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
-b {DMLlist} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-DML.txt

In [123]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-DML.txt

NC_035780.1	8833124	8833126	NC_035780.1	8833124	8833126	60
NC_035780.1	55484705	55484707	NC_035780.1	55484705	55484707	-52
NC_035780.1	55484705	55484707	NC_035780.1	55484705	55484707	-52
NC_035780.1	58135767	58135769	NC_035780.1	58135767	58135769	74
NC_035781.1	7626510	7626512	NC_035781.1	7626510	7626512	-56
NC_035781.1	7626510	7626512	NC_035781.1	7626510	7626512	-56
NC_035781.1	30789623	30789625	NC_035781.1	30789623	30789625	-57
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035782.1	4729348	4729350	NC_035782.1	4729348	4729350	55
NC_035782.1	4729348	4729350	NC_035782.1	4729348	4729350	55


In [124]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-DML.txt

      67 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-DML.txt


In [125]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed \
-b {DMLlist} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-DML.txt

In [126]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-DML.txt

NC_035780.1	1882691	1882693	NC_035780.1	1882691	1882693	64
NC_035781.1	20126029	20126031	NC_035781.1	20126029	20126031	-52
NC_035781.1	28992818	28992820	NC_035781.1	28992818	28992820	52
NC_035781.1	30062222	30062224	NC_035781.1	30062222	30062224	60
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53


In [127]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-DML.txt

      49 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-DML.txt


#### Hypermethylated DML

In [128]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
-b {hyperDML} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypermethylated-DML.txt

In [129]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypermethylated-DML.txt

NC_035780.1	8833124	8833126	NC_035780.1	8833124	8833126	60
NC_035780.1	58135767	58135769	NC_035780.1	58135767	58135769	74
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035782.1	4729348	4729350	NC_035782.1	4729348	4729350	55
NC_035782.1	4729348	4729350	NC_035782.1	4729348	4729350	55
NC_035783.1	19545473	19545475	NC_035783.1	19545473	19545475	50
NC_035783.1	45068802	45068804	NC_035783.1	45068802	45068804	52
NC_035783.1	57931740	57931742	NC_035783.1	57931740	57931742	50
NC_035784.1	26800339	26800341	NC_035784.1	26800339	26800341	66
NC_035784.1	57514692	57514694	NC_035784.1	57514692	57514694	53


In [130]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypermethylated-DML.txt

      44 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypermethylated-DML.txt


In [131]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed \
-b {hyperDML} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypermethylated-DML.txt

In [132]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypermethylated-DML.txt

NC_035780.1	1882691	1882693	NC_035780.1	1882691	1882693	64
NC_035781.1	28992818	28992820	NC_035781.1	28992818	28992820	52
NC_035781.1	30062222	30062224	NC_035781.1	30062222	30062224	60
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	31150010	31150012	NC_035781.1	31150010	31150012	53
NC_035781.1	49000882	49000884	NC_035781.1	49000882	49000884	61


In [133]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypermethylated-DML.txt

      34 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypermethylated-DML.txt


#### Hypomethylated DML

In [134]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
-b {hypoDML} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypomethylated-DML.txt

In [135]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypomethylated-DML.txt

NC_035780.1	55484705	55484707	NC_035780.1	55484705	55484707	-52
NC_035780.1	55484705	55484707	NC_035780.1	55484705	55484707	-52
NC_035781.1	7626510	7626512	NC_035781.1	7626510	7626512	-56
NC_035781.1	7626510	7626512	NC_035781.1	7626510	7626512	-56
NC_035781.1	30789623	30789625	NC_035781.1	30789623	30789625	-57
NC_035782.1	6685343	6685345	NC_035782.1	6685343	6685345	-68
NC_035782.1	6685349	6685351	NC_035782.1	6685349	6685351	-50
NC_035782.1	6897406	6897408	NC_035782.1	6897406	6897408	-53
NC_035782.1	72205396	72205398	NC_035782.1	72205396	72205398	-55
NC_035782.1	72205396	72205398	NC_035782.1	72205396	72205398	-55


In [137]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypomethylated-DML.txt

      23 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-Hypomethylated-DML.txt


In [138]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed \
-b {hypoDML} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypomethylated-DML.txt

In [139]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypomethylated-DML.txt

NC_035781.1	20126029	20126031	NC_035781.1	20126029	20126031	-52
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51
NC_035784.1	1062719	1062721	NC_035784.1	1062719	1062721	-51


In [140]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypomethylated-DML.txt

      15 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-Hypomethylated-DML.txt


#### DMR

In [38]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-Upstream-Flanks.bed \
-b {DMRlist} \
> 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-UpstreamFlanks-DMR.txt

In [39]:
!head 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-UpstreamFlanks-DMR.txt

NC_035780.1	Gnomon	mRNA	46044701	46044800	.	-	.	ID=rna4633;Parent=gene2726;Dbxref=GeneID:111110504,Genbank:XM_022447031.1;Name=XM_022447031.1;gbkey=mRNA;gene=LOC111110504;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1 sample with support for all annotated introns;product=uncharacterized LOC111110504%2C transcript variant X1;transcript_id=XM_022447031.1	NC_035780.1	46044700	46044800	DMR	-53
NC_035780.1	Gnomon	mRNA	46044701	46044800	.	-	.	ID=rna4634;Parent=gene2726;Dbxref=GeneID:111110504,Genbank:XM_022447040.1;Name=XM_022447040.1;gbkey=mRNA;gene=LOC111110504;model_evidence=Supporting evidence includes similarity to: 1 Protein%2C and 100%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 1 sample with support for all annotated introns;product=uncharacterized LOC111110504%2C transcript variant X2;transcript_id=XM_022447040.1	NC_035780.1	46044700	460

In [40]:
!wc -l 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-UpstreamFlanks-DMR.txt

      12 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-UpstreamFlanks-DMR.txt


In [41]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-Downstream-Flanks.bed \
-b {DMRlist} \
> 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-DownstreamFlanks-DMR.txt

In [42]:
!head 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-DownstreamFlanks-DMR.txt

NC_035780.1	Gnomon	mRNA	573701	573800	.	+	.	ID=rna48;Parent=gene35;Dbxref=GeneID:111114201,Genbank:XM_022452489.1;Name=XM_022452489.1;Note=The sequence of the model RefSeq transcript was modified relative to this genomic sequence to represent the inferred CDS: inserted 2 bases in 2 codons;exception=unclassified transcription discrepancy;gbkey=mRNA;gene=LOC111114201;model_evidence=Supporting evidence includes similarity to: 4 Proteins%2C and 99%25 coverage of the annotated genomic feature by RNAseq alignments%2C including 9 samples with support for all annotated introns;product=vacuolar protein sorting-associated protein 13B-like;transcript_id=XM_022452489.1	NC_035780.1	573700	573800	DMR	52
NC_035780.1	Gnomon	mRNA	33945901	33946000	.	-	.	ID=rna3547;Parent=gene2078;Dbxref=GeneID:111131167,Genbank:XM_022478582.1;Name=XM_022478582.1;gbkey=mRNA;gene=LOC111131167;model_evidence=Supporting evidence includes similarity to: 5 Proteins%2C and 100%25 coverage of the annotated genomic feature by R

In [43]:
!wc -l 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-DownstreamFlanks-DMR.txt

      18 2018-11-14-Flanking-Analysis/2018-11-15-mRNA-100bp-DownstreamFlanks-DMR.txt


#### CG motifs

In [141]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
-b {CGMotifList} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-CGmotif.txt

In [142]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-CGmotif.txt

NC_035780.1	27969	27971	NC_035780.1	27969	27971	CG_motif
NC_035780.1	27979	27981	NC_035780.1	27979	27981	CG_motif
NC_035780.1	28081	28083	NC_035780.1	28081	28083	CG_motif
NC_035780.1	28130	28132	NC_035780.1	28130	28132	CG_motif
NC_035780.1	28147	28149	NC_035780.1	28147	28149	CG_motif
NC_035780.1	28169	28171	NC_035780.1	28169	28171	CG_motif
NC_035780.1	28209	28211	NC_035780.1	28209	28211	CG_motif
NC_035780.1	28211	28213	NC_035780.1	28211	28213	CG_motif
NC_035780.1	28228	28230	NC_035780.1	28228	28230	CG_motif
NC_035780.1	28308	28310	NC_035780.1	28308	28310	CG_motif


In [143]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-CGmotif.txt

 1287046 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-UpstreamFlanks-CGmotif.txt


In [144]:
! {bedtoolsDirectory}intersectBed \
-wb \
-a 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Downstream-Flanks.bed \
-b {CGMotifList} \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-CGmotif.txt

In [145]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-CGmotif.txt

NC_035780.1	33407	33409	NC_035780.1	33407	33409	CG_motif
NC_035780.1	33451	33453	NC_035780.1	33451	33453	CG_motif
NC_035780.1	33480	33482	NC_035780.1	33480	33482	CG_motif
NC_035780.1	33637	33639	NC_035780.1	33637	33639	CG_motif
NC_035780.1	33646	33648	NC_035780.1	33646	33648	CG_motif
NC_035780.1	33783	33785	NC_035780.1	33783	33785	CG_motif
NC_035780.1	33796	33798	NC_035780.1	33796	33798	CG_motif
NC_035780.1	34283	34285	NC_035780.1	34283	34285	CG_motif
NC_035780.1	34310	34312	NC_035780.1	34310	34312	CG_motif
NC_035780.1	34321	34323	NC_035780.1	34321	34323	CG_motif


In [146]:
!wc -l 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-CGmotif.txt

 1285232 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-100bp-DownstreamFlanks-CGmotif.txt


### 6b. No overlaps

I also want to count the number of DML or DMR that do not overlap with any features (i.e. DML and DMR in unannotated intergenic regions). To do this, I'll use the `-v` argument in `bedtools`, which reports "those entries in A that have no overlap in B." I can specify multiple files with `-b`. I'll use exons, introns, transposable elements identified using all species, and putative promoter regions (upstream flanks).

#### DML

In [147]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {DMLlist} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed  \
| wc -l
!echo "DML do not overlap with exons, introns, transposable elements (all), or putative promoters"

      15
DML do not overlap with exons, introns, transposable elements (all), or putative promoters


In [148]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {DMLlist} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
> 2019-05-29-No-Overlap-DML.txt

In [149]:
!head 2019-05-29-No-Overlap-DML.txt

NC_035781.1	20620123	20620125	57
NC_035781.1	30062222	30062224	60
NC_035781.1	39583208	39583210	-50
NC_035781.1	50711254	50711256	-71
NC_035782.1	58675230	58675232	52
NC_035782.1	65377028	65377030	51
NC_035784.1	2011997	2011999	-60
NC_035784.1	45667412	45667414	56
NC_035784.1	53515949	53515951	50
NC_035784.1	81666532	81666534	-65


#### Hypermethylated DML

In [150]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {hyperDML} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed  \
| wc -l
!echo "hypermethylated DML do not overlap with exons, introns, transposable elements (all), or putative promoters"

       9
hypermethylated DML do not overlap with exons, introns, transposable elements (all), or putative promoters


In [151]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {hyperDML} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
> 2019-05-29-No-Overlap-Hypermethylated-DML.txt

In [152]:
!head 2019-05-29-No-Overlap-Hypermethylated-DML.txt

NC_035781.1	20620123	20620125	57
NC_035781.1	30062222	30062224	60
NC_035782.1	58675230	58675232	52
NC_035782.1	65377028	65377030	51
NC_035784.1	45667412	45667414	56
NC_035784.1	53515949	53515951	50
NC_035785.1	31238802	31238804	59
NC_035787.1	42603398	42603400	57
NC_035787.1	44016221	44016223	70


#### Hypomethylated DML

In [154]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {hypoDML} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed  \
| wc -l
!echo "hypomethylated DML do not overlap with exons, introns, transposable elements (all), or putative promoters"

       6
hypomethylated DML do not overlap with exons, introns, transposable elements (all), or putative promoters


In [155]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {hypoDML} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
> 2019-05-29-No-Overlap-Hypomethylated-DML.txt

In [156]:
!head 2019-05-29-No-Overlap-Hypomethylated-DML.txt

NC_035781.1	39583208	39583210	-50
NC_035781.1	50711254	50711256	-71
NC_035784.1	2011997	2011999	-60
NC_035784.1	81666532	81666534	-65
NC_035787.1	42755937	42755939	-54
NC_035788.1	78353418	78353420	-75


#### CG motifs

In [157]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {CGMotifList} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed  \
| wc -l
!echo "CG motifs do not overlap with exons, introns, transposable elements (all), or putative promoters"

 4528757
CG motifs do not overlap with exons, introns, transposable elements (all), or putative promoters


In [158]:
! {bedtoolsDirectory}intersectBed \
-v \
-a {CGMotifList} \
-b {exonList} {intronList} {transposableElementsAll} 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Upstream-Flanks.bed \
> 2019-05-29-No-Overlap-CGmotifs.txt

In [159]:
!head 2019-05-29-No-Overlap-CGmotifs.txt

NC_035780.1	28	30	CG_motif
NC_035780.1	54	56	CG_motif
NC_035780.1	75	77	CG_motif
NC_035780.1	93	95	CG_motif
NC_035780.1	103	105	CG_motif
NC_035780.1	116	118	CG_motif
NC_035780.1	134	136	CG_motif
NC_035780.1	159	161	CG_motif
NC_035780.1	209	211	CG_motif
NC_035780.1	224	226	CG_motif


### 6c. `closest`

[`bedtools closest`](https://bedtools.readthedocs.io/en/latest/content/tools/closest.html) will find the nearest DML or DMR to an gene. If the closest feature is not overlapping, I'll get the distance to the next feature. If the closest feature is overlapping, the distance would be zero. I will use the following code:

1. Path to `closestBed`
3. -a: Path to gene list
4. -b: Specify either DML, DMR, or CG motif file. The CG motif file will be used as a background for the DML and DMR
6. -t all: In case of a tie, report all matches
7. -D ref: Report distance to A in an extra column. Use negative distances to report upstream features with respect to the reference genome. B features with a lower (start, stop) are upstream.
8. ">" filename: Redirect output to a .txt file

In [160]:
! {bedtoolsDirectory}closestBed \
-a {geneList} \
-b {DMLlist} \
-t all \
-D ref \
> 2019-05-29-Flanking-Analysis/2019-05-29-Genes-Closest-NoOverlap-DMLs.txt

In [162]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-Genes-Closest-NoOverlap-DMLs.txt

NC_035780.1	13578	14594	NC_035780.1	401630	401632	53	387037
NC_035780.1	28961	33324	NC_035780.1	401630	401632	53	368307
NC_035780.1	43111	66897	NC_035780.1	401630	401632	53	334734
NC_035780.1	85606	95254	NC_035780.1	401630	401632	53	306377
NC_035780.1	99840	106460	NC_035780.1	401630	401632	53	295171
NC_035780.1	108305	110077	NC_035780.1	401630	401632	53	291554
NC_035780.1	151859	157536	NC_035780.1	401630	401632	53	244095
NC_035780.1	163809	183798	NC_035780.1	401630	401632	53	217833
NC_035780.1	164820	166793	NC_035780.1	401630	401632	53	234838
NC_035780.1	169468	170178	NC_035780.1	401630	401632	53	231453


In [163]:
! {bedtoolsDirectory}closestBed \
-a {geneList} \
-b {DMRlist} \
-t all \
-D ref \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Closest-NoOverlap-DMRs.txt

/bin/sh: {bedtoolsDirectory}closestBed: command not found


In [45]:
!head 2019-05-29-Flanking-Analysis/2018-11-14-mRNA-Closest-NoOverlap-DMRs.txt

In [164]:
! {bedtoolsDirectory}closestBed \
-a {geneList} \
-b {CGMotifList} \
-t all \
-D ref \
> 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Closest-NoOverlap-CGmotifs.txt

ERROR: Database file C_virginica-3.0_CG-motif.bed contains chromosome NC_007175.2, but the query file does not.
       Please re-reun with the -g option for a genome file.
       See documentation for details.


In [165]:
!head 2019-05-29-Flanking-Analysis/2019-05-29-mRNA-Closest-NoOverlap-CGmotifs.txt

NC_035780.1	13578	14594	NC_035780.1	13597	13599	CG_motif	0
NC_035780.1	13578	14594	NC_035780.1	13651	13653	CG_motif	0
NC_035780.1	13578	14594	NC_035780.1	13725	13727	CG_motif	0
NC_035780.1	13578	14594	NC_035780.1	14144	14146	CG_motif	0
NC_035780.1	13578	14594	NC_035780.1	14430	14432	CG_motif	0
NC_035780.1	13578	14594	NC_035780.1	14453	14455	CG_motif	0
NC_035780.1	28961	33324	NC_035780.1	28992	28994	CG_motif	0
NC_035780.1	28961	33324	NC_035780.1	29001	29003	CG_motif	0
NC_035780.1	28961	33324	NC_035780.1	29028	29030	CG_motif	0
NC_035780.1	28961	33324	NC_035780.1	29180	29182	CG_motif	0
